# Увеличиваем датасет с помощью синонимов.

## Импорты

In [ ]:
# ! pip install stanza

     |████████████████████████████████| 432 kB 5.4 MB/s 
     |████████████████████████████████| 170 kB 43.3 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=7166ac3fe6126f5464d042e24e72d20235fa9469fbc62458767a573230ab3e38
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [10]:
import stanza

In [11]:
stanza.download("ru")

2021-12-26 22:50:09 INFO: Downloading default packages for language: ru (Russian)...
2021-12-26 22:50:11 INFO: File exists: /root/stanza_resources/ru/default.zip.
2021-12-26 22:50:18 INFO: Finished downloading models and saved to /root/stanza_resources.


In [12]:
nlp = stanza.Pipeline("ru")

2021-12-26 22:50:18 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |
| ner       | wikiner   |

2021-12-26 22:50:18 INFO: Use device: cpu
2021-12-26 22:50:18 INFO: Loading: tokenize
2021-12-26 22:50:18 INFO: Loading: pos
2021-12-26 22:50:18 INFO: Loading: lemma
2021-12-26 22:50:18 INFO: Loading: depparse
2021-12-26 22:50:19 INFO: Loading: ner
2021-12-26 22:50:20 INFO: Done loading processors!


In [2]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download("stopwords")
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("english")
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

import gensim
from gensim.models import KeyedVectors

from string import punctuation
import re
from tqdm import tqdm_notebook
from collections import Counter
import pandas as pd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Генерим новые тексты

Читаем ревью

In [4]:
with open ('train_split_reviews.txt', encoding='utf-8') as f:
    data = f.readlines()

Читаем тексты

In [5]:
with open ('train_split_aspects.txt', encoding='utf-8') as f:
    data_a = f.readlines()

In [6]:
aspects_lemmas = [lemmatizer.lemmatize(a.split('\t')[2]) for a in data_a]

In [7]:
aspects = list(set(aspects_lemmas))

Импортируем модель

In [ ]:
!wget http://vectors.nlpl.eu/repository/20/187.zip

--2021-12-26 22:42:20--  http://vectors.nlpl.eu/repository/20/187.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2692389554 (2.5G) [application/zip]
Saving to: ‘187.zip’

187.zip             100%[===================>]   2.51G  23.1MB/s    in 3m 11s  

2021-12-26 22:45:33 (13.4 MB/s) - ‘187.zip’ saved [2692389554/2692389554]



In [ ]:
!unzip 187.zip

Archive:  187.zip
  inflating: meta.json               
  inflating: model.model             
  inflating: model.model.vectors_ngrams.npy  
  inflating: model.model.vectors.npy  
  inflating: model.model.vectors_vocab.npy  
  inflating: README                  


In [ ]:
!pip install gensim==3.8.0

     |████████████████████████████████| 24.2 MB 1.5 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [8]:
model = KeyedVectors.load("model.model")

Для каждого выделенного аспекта запоминаем часть речи вершины аспекта, затем в синонимах оставляем только те, которые совпадают по части речи с вершиной аспекта. Лемматизируем синонимы, проверяем, что не дублируемся.



In [13]:
aspects_syn = {}
for word in tqdm_notebook(aspects):
    for t in nlp(word).sentences[0].words:
        if t.deprel == 'root':
            pos = t.upos
    aspects_syn[word] = []
    if word in model:
        for (w, score) in model.wv.most_similar(word, topn=10):
            nlpw = nlp(w).sentences[0].words[0]
            if nlpw.lemma not in aspects_syn[word] and nlpw.upos == pos:
                    aspects_syn[word].append(nlpw.lemma)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/1509 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [14]:
reviews_dict = {}
for sent in data:
    sent = sent.split('\t')
    reviews_dict[sent[0]] = sent[-1]

In [15]:
gen_reviews_dict = {}

Заменяем аспекты на синонимы, если это возможно, если нет - оставляем вместо синонима аспект. Генерим предложения и составляем размеченные аспекты.

In [16]:
to_change = []
for k in tqdm_notebook(reviews_dict.keys()):
    asps = []
    for line in data_a:
        if line.split('\t')[0] == k:
            asps.append(line)
    for i, a in enumerate(asps):
        sent_id, cat, asp, b, e, sen = a.split('\t')
        if len(aspects_syn[asp]) == 0:
            try:
                syn = aspects_syn[asp.lower()][0]
            except:
                syn = asp
        else:
            try:
                syn = aspects_syn[asp.lower()][0]
            except:
                syn = aspects_syn[asp][0]
            if syn == asp.lower():
                try:
                    syn = aspects_syn[asp.lower()][1]
                except:
                    syn = aspects_syn[asp][1]
            if syn == nlp(asp).sentences[0].words[0].lemma:
                try:
                    syn = aspects_syn[asp][1]
                except:
                    pass
        l = [sent_id+'a', cat, syn]
        if i == 0:
            sent = reviews_dict[sent_id]
        else:
            sent = gen_reviews_dict[sent_id+'a']
        sent_change = re.sub(asp, syn, sent)
        l.append(str(sent_change.find(syn)))
        l.append(str(sent_change.find(syn)+len(syn)))
        l.append(sen)
        to_change.append(l)
        gen_reviews_dict[sent_id+'a'] = sent_change

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/213 [00:00<?, ?it/s]

In [17]:
reviews_dict['11388']

'Несколько раз посещали с друзьями данный ресторан! В общем очень даже неплохо! Интерьер милый, всего в меру! Кухня понравилась! Последний раз были 21 января, отлично посидели! Была небольшая компания 5 человек. Очень понравился музыкальный коллектив, хорошая музыка! По обслуживанию претензий нет, хорошо встретили, проводили за столик (забронировали заранее в VIP зоне, стоимость 400 руб.). Официанты не заставляли себя долго ждать, очень все вежливо и обходительно! Вполне прилично место!\n'

In [18]:
gen_reviews_dict['11388a']

'Несколько раз посещали с друзьями данный ресторанут! В общем очень даже неплохо! экстерьер милый, всего в меру! кухно понравилась! Последний раз были 21 января, отлично посидели! Была небольшая компания 5 человек. Очень понравился поп-музыкальность, хорошая поп-музыка! По медобслуживание претензий нет, хорошо повстречать, выпроводить (забронировали заранее в VIP зоне, стоимость 400 руб.). пант не заставляли себя долго ждать, очень все вежливо и обходительно! Вполне прилично местотец!\n'

Записываем в файлики.

In [19]:
with open('more_train_split_reviews.txt', 'a', encoding='utf-8') as f:
    for line in data:
        f.write(line+'\r\n')
    for key, value in gen_reviews_dict.items():
        f.write(key+'\t'+value+'\n')

In [20]:
with open('more_train_split_aspects.txt', 'a', encoding='utf-8') as f:
    for line in data_a:
        f.write(line)
    for k in to_change:
        f.write('\t'.join(k))

## Dataset statistics

Смотрим объём корпуса

In [21]:
with open ('train_reviews.txt', encoding='utf-8') as f:
    data = f.readlines()
len(data)

284

Средняя длина отзыва.

In [22]:
def reviews_tuple(raw):
    s_raw = raw.split('\t')
    return s_raw

In [23]:
reviews = [reviews_tuple(r) for r in data]
texts = [r[-1] for r in reviews]
lens = [len(i.split()) for i in texts]
sum = 0
for l in lens:
    sum+=l
sum/len(lens)

129.46478873239437

Среднее количество аспектов в тексте.

In [24]:
num_aspects = []
for k in reviews_dict.keys():
    asps = 0
    for line in data_a:
        if line.split('\t')[0] == k:
            asps+=1
    num_aspects.append(asps)

sum = 0
for l in num_aspects:
    sum+=l
sum/len(num_aspects)

16.708920187793428

Распределение оценок по категориям.

In [25]:
with open ('train_split_cats.txt', encoding='utf-8') as f:
    data_c = f.readlines()

In [26]:
food = []
interior = []
price=[]
whole=[]
service=[]
for l in data_c:
    c = l.split('\t')[1]
    s = l.split('\t')[-1].replace('\n', '')
    if c=='Food':
        food.append(s)
    elif c=='Interior':
        interior.append(s)
    elif c=='Price':
        price.append(s)
    elif c=='Whole':
        whole.append(s)
    elif c=='Service':
        service.append(s)

In [27]:
print('Food: positive = ', Counter(food)['positive']/len(food)*100, '%, negative = ', Counter(food)['negative']/len(food)*100, '%.')
print('Interior: positive = ', Counter(interior)['positive']/len(interior)*100, '%, negative = ', Counter(interior)['negative']/len(interior)*100, '%.')
print('Price: positive = ', Counter(price)['positive']/len(price)*100, '%, negative = ', Counter(price)['negative']/len(price)*100, '%.')
print('Whole: positive = ', Counter(whole)['positive']/len(whole)*100, '%, negative = ', Counter(whole)['negative']/len(whole)*100, '%.')
print('Service: positive = ', Counter(service)['positive']/len(service)*100, '%, negative = ', Counter(service)['negative']/len(service)*100, '%.')

Food: positive =  71.36150234741784 %, negative =  7.981220657276995 %.
Interior: positive =  65.25821596244131 %, negative =  4.225352112676056 %.
Price: positive =  21.12676056338028 %, negative =  9.859154929577464 %.
Whole: positive =  70.4225352112676 %, negative =  15.023474178403756 %.
Service: positive =  62.91079812206573 %, negative =  17.84037558685446 %.


In [28]:
with open('more_train_split_cats.txt', 'a', encoding='utf-8') as f:
    for line in data_c:
        f.write(line)
    for k in to_change:
        f.write('\t'.join([k[0], k[1], k[-1]]))